In [3]:
import os
import sys

import warnings
warnings.filterwarnings("ignore")

os.environ['environment'] = 'interactive'
os.environ['UTILS_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/python-utils-ak'
os.environ['PROJECT_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat' 
os.environ['SQLITE_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat/data.sqlite' 
sys.path.append(os.environ['UTILS_PATH'])
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

WORKING IN INTERACTIVE MODE


In [4]:
from utils_ak.block_tree import *
from utils_ak.iteration import *
from utils_ak.builtin import *

from app.schedule_maker.models import *

In [5]:
values = [['a', '0.2'], ['b', '0.2'], ['b', '0.2']]

In [6]:
df = pd.DataFrame(values)
df

,0,1
0,a,0.2
1,b,0.2
2,b,0.2


In [8]:
from collections import defaultdict


In [43]:
df = pd.DataFrame([['A', 'a', 'a1'],
                   ['A', 'a', 'a2'],
                   ['B', 'b', 'b1'],
                   ['B', 'b', 'b2'],
                   ['A', 'c', 'c1'],
                   ['A', 'c', 'c2']],
                  columns=['col1', 'col2', 'col3'])
df

,col1,col2,col3
0,A,a,a1
1,A,a,a2
2,B,b,b1
3,B,b,b2
4,A,c,c1
5,A,c,c2


In [44]:
def df_to_tree(df, recursive=True):
    if len(df.columns) == 1:
        return df[df.columns[0]].tolist()
        
    res = {}
    
    for value, grp in df.groupby(df.columns[0]):
        grp.pop(df.columns[0])
        if recursive:
            res[value] = df_to_tree(grp, recursive=True)
        else:
            res[value] = grp
    return res

In [60]:
def df_to_ordered_tree(df, recursive=True):
    if len(df.columns) == 1:
        return df[df.columns[0]].tolist()
        
    res = []
    pre_res = []
    
    col = df.columns[0]
    mark_consecutive_groups(df, col, '__key')
    
    for value, grp in df.groupby([col, '__key']):
        grp.pop(col)
        grp.pop('__key')
        if recursive:
            child = df_to_tree(grp, recursive=True)
        else:
            child = grp
        pre_res.append((value[0], value[1], child))
    pre_res = list(sorted(pre_res, key=lambda obj: df[(df[col] == obj[0]) & (df['__key'] == obj[1])].iloc[0]['__key']))
    return [(x[0], x[2]) for x in pre_res]

In [61]:
df_to_tree(df, True)

{'A': {'a': {'a1': [1], 'a2': [1]}, 'c': {'c1': [3], 'c2': [3]}},
 'B': {'b': {'b1': [2], 'b2': [2]}}}

In [62]:
df_to_ordered_tree(df)

[('A', {'a': ['a1', 'a2']}),
 ('B', {'b': ['b1', 'b2']}),
 ('A', {'c': ['c1', 'c2']})]

In [29]:
from utils_ak.pandas import *

In [30]:
mark_consecutive_groups(df, 'col1', 'new_col')

In [31]:
df

,col1,col2,col3,new_col
0,A,a,a1,1
1,A,a,a2,1
2,B,b,b1,2
3,B,b,b2,2
4,A,c,c1,3
5,A,c,c2,3
